#### Client의 행동을 따라가보자
- Client가 어떻게 행동하는지 데이터 살펴보기

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [6]:
#데이터가져오기
Product = pd.read_csv("C:/Users/lsh93/Desktop/data/Product.csv")
Search1 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search1.csv")
Search2 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search2.csv")
Custom = pd.read_csv("C:/Users/lsh93/Desktop/data/Custom.csv")
Session = pd.read_csv("C:/Users/lsh93/Desktop/data/Session.csv")
Master = pd.read_csv("C:/Users/lsh93/Desktop/data/Master.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#구매데이터의 client 수 != 고객데이터의 client 수 
#구매데이터의 client 수 != 검색데이터의 client 수
print('Product shape: ',Product.shape)
print('Custom shape: ',Custom.shape)
print('Search1 shape: ',Search1.shape)
print('Session shape: ',Session.shape)
print()
print('구매에 참여한 client 수: ',len(Product['CLNT_ID'].value_counts()))
print('고객데이터에 있는 unique한 client 수: ',len(Custom['CLNT_ID'].value_counts()))
print('세션에 참여한 unique한 client 수: ',len(Session['CLNT_ID'].value_counts()))
print('검색에 참여한 client 수: ', len(Search1['CLNT_ID'].value_counts()))

Product shape:  (5024906, 8)
Custom shape:  (671679, 3)
Search1 shape:  (2884943, 4)
Session shape:  (2712907, 9)

구매에 참여한 client 수:  922737
고객데이터에 있는 unique한 client 수:  671679
세션에 참여한 unique한 client 수:  922737
검색에 참여한 client 수:  511477


- 세션에 참여한 client 수와 구매에 참여한 client 수는 동일하다 
- 상품을 구매했을지라도 검색을 하지 않은 client들이 있다 ) 922737 - 511477 만큼
- 고객데이터에 모든 client들의 정보가 있는 것은 아니다 

In [4]:
temp1 = pd.merge(Search1, Product, how = 'right' , on = 'CLNT_ID')

In [5]:
print(len(temp1['CLNT_ID'].value_counts()))

922737


In [6]:
temp2 = pd.merge(Custom, Product, how = 'right' , on = 'CLNT_ID')

In [7]:
print(len(temp1['CLNT_ID'].value_counts()))

922737


In [8]:
#null값 체크 --> 이건 unique한 건 아니지만 그래도 null값이 있다는 걸 의미함 
print(temp2['CLNT_AGE'].isnull().sum().sum())

1036319


- Client ID, 날짜(세션일자), 세션 ID, HIT번호, 검색키워드, 검색건수, 상품명, 구매건수 

- CLNT_ID(Product), SESS_DT(Session), SESS_ID(세션ID), HITS_SEQ(Product)
- KWD_NM(Search1), SEARCH_CNT(Search1), PD_NM(Master), PD_BUY_CT(Product)

#### Product에 있는 Client 만을 기준으로, 행동을 나열함 
#### 주의: 구매에 참여한 모든 client가 검색을 한 것은 아님

In [7]:
#구매에 참여한 client와 세션에 참여한 client의 수는 같으므로 inner join
data1 = pd.merge(Product, Session)
print(len(data1['CLNT_ID'].value_counts()))

922737


In [8]:
#상품정보를 추가해준다
data2 = pd.merge(data1, Master, how = 'left', on = 'PD_C')
print(len(data1['CLNT_ID'].value_counts()))

922737


In [11]:
#세션과 구매에 참여한 client를 기준으로 data2과 Search1을 left outer join 해 준다 
#세션과 구매에 참여한 client일지라도 검색을 하지 않은 client가 존재한다
data2 = data2[['CLNT_ID','SESS_ID','HITS_SEQ','SESS_DT','TOT_PAG_VIEW_CT','PD_NM','PD_BUY_CT','ZON_NM']]

In [14]:
idx = data2['CLNT_ID'] == 4139680
data2[idx]

,CLNT_ID,SESS_ID,HITS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,PD_NM,PD_BUY_CT,ZON_NM
0,4139680,7605037,12,20180526,21.0,크러쉬드 립칼라 - 플럼,1,Seoul
1,4139680,7605037,12,20180526,21.0,[한정판] 크러쉬드 립칼라 2.0 - 몰리 와우,1,Seoul
2,4139680,7605037,28,20180526,21.0,[한정판] 크러쉬드 립칼라 2.0 - 몰리 와우,1,Seoul
3,4139680,7605037,28,20180526,21.0,크러쉬드 립칼라 - 플럼,1,Seoul
4,4139680,7605037,12,20180526,21.0,크러쉬드 립칼라 - 플럼,1,Seoul
5,4139680,7605037,28,20180526,21.0,크러쉬드 립칼라 - 플럼,1,Seoul


In [28]:
#Session id가 양쪽 테이블에서 다르기 때문에(구매랑 검색이랑 다른 세션 아이디 가질 수도 그래서 outer로 해야 )
Client_Behavior = pd.merge(data2, Search1, how = 'outer', on = 'CLNT_ID')

In [29]:
Client_Behavior.head()

,CLNT_ID,SESS_ID_x,HITS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,PD_NM,PD_BUY_CT,ZON_NM,SESS_ID_y,KWD_NM,SEARCH_CNT
0,4139680,7605037,12,20180526,21.0,크러쉬드 립칼라 - 플럼,1,Seoul,NaN,NaN,NaN
1,4139680,7605037,12,20180526,21.0,[한정판] 크러쉬드 립칼라 2.0 - 몰리 와우,1,Seoul,NaN,NaN,NaN
2,4139680,7605037,28,20180526,21.0,[한정판] 크러쉬드 립칼라 2.0 - 몰리 와우,1,Seoul,NaN,NaN,NaN
3,4139680,7605037,28,20180526,21.0,크러쉬드 립칼라 - 플럼,1,Seoul,NaN,NaN,NaN
4,4139680,7605037,12,20180526,21.0,크러쉬드 립칼라 - 플럼,1,Seoul,NaN,NaN,NaN


- client id = 4140076 일 경우

In [31]:
idx = Client_Behavior['CLNT_ID'] == 4140076
Client_Behavior[idx]

,CLNT_ID,SESS_ID_x,HITS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,PD_NM,PD_BUY_CT,ZON_NM,SESS_ID_y,KWD_NM,SEARCH_CNT
6,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,10342512.0,크록스,1.0
7,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,5338950.0,여성백팩,2.0
8,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,7803570.0,닥스여아,1.0
9,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,7803570.0,프라다,1.0
10,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,4756252.0,데코뷰,1.0
11,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,6526651.0,나이키슬리퍼,1.0
12,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,6526651.0,프레드페리 남성,2.0
13,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,6526651.0,게스 남성,1.0
14,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,8326529.0,아디다스반바지,1.0
15,4140076,10189797,13,20180413,18.0,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,2,Gyeonggi-do,4756252.0,뷰,1.0


In [32]:
#Session이랑 Search1, Product에서 같은 client id에 대해서 SESS_ID비교,,, 뭔가 이상해서 
idx = Session['CLNT_ID'] == 4140076
Session[idx]

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
880512,4140076,10767546,139,20180404,19.0,467,mobile,Seoul,Seoul
880513,4140076,8891563,224,20180505,19.0,172,mobile,Gyeonggi-do,Icheon-si
1464484,4140076,6517872,273,20180613,179.0,"1,563",mobile,Gyeonggi-do,Icheon-si
1464485,4140076,9147546,213,20180501,473.0,"8,164",mobile,Gyeonggi-do,Icheon-si
1464486,4140076,2051362,304,20180827,78.0,"1,992",mobile,Seoul,Seoul
1464487,4140076,8326529,229,20180514,199.0,"2,055",mobile,Gyeonggi-do,Icheon-si
1464488,4140076,7782189,241,20180523,128.0,"2,066",mobile,Gyeonggi-do,Icheon-si
1464489,4140076,10760242,140,20180404,26.0,54,mobile,Gyeonggi-do,Icheon-si
1464490,4140076,10574005,146,20180407,257.0,"1,201",mobile,Seoul,Seoul
1464491,4140076,10584304,145,20180407,79.0,"1,400",mobile,Gyeonggi-do,Icheon-si


In [33]:
#Session이랑 Search1, Product에서 같은 client id에 대해서 SESS_ID비교,,, 뭔가 이상해서 
idx = Search1['CLNT_ID'] == 4140076
Search1[idx]

,CLNT_ID,SESS_ID,KWD_NM,SEARCH_CNT
365805,4140076,10342512,크록스,1
365806,4140076,5338950,여성백팩,2
434665,4140076,7803570,닥스여아,1
434666,4140076,7803570,프라다,1
1861286,4140076,4756252,데코뷰,1
1861287,4140076,6526651,나이키슬리퍼,1
1861288,4140076,6526651,프레드페리 남성,2
1861289,4140076,6526651,게스 남성,1
1861290,4140076,8326529,아디다스반바지,1
1861291,4140076,4756252,뷰,1


In [34]:
idx = Product['CLNT_ID'] == 4140076
Product[idx]

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
1,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2
132425,4140076,10342512,23,422358,선택:02.204143-02S|사이즈:M5W7(240) / 1개,크록스,"39,900",1
647279,4140076,6526651,22,196272,색상:선택3)248(네이비)|사이즈:L(105) / 1개,프레드페리,"47,600",1
673008,4140076,8326529,199,305965,사이즈:115_3XL / 1개,아디다스(의류),"46,700",1
720869,4140076,8891563,19,305965,사이즈:115_3XL / 1개,아디다스(의류),"46,700",1
783221,4140076,6526651,193,196272,색상:선택2)102(블랙)|사이즈:XL(110) / 1개,프레드페리,"47,600",1
784254,4140076,6591360,149,196272,색상:선택2)102(블랙)|사이즈:XL(110) / 2개,프레드페리,"47,600",2
844514,4140076,6526651,22,365025,색상:WH|사이즈:165 / 1개,닥스 키즈,"48,300",1
1062425,4140076,4756252,147,794908,1개,네이쳐리빙,"7,200",1
1168519,4140076,10342512,23,403243,사이즈:M9 / 1개,크록스,"57,000",1
